In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [15]:
def get_dictionaries(ratingsdf,usercol,itemcol,ratingcol):
    users = list(set(ratingsdf[usercol].values))
    items = list(set(ratingsdf[itemcol].values))
    ratings_by_user = {user_id:{} for user_id in users}
    users_by_item = {item_id:[] for item_id in items}
    for review_index in ratingsdf.index:
        user_id = ratingsdf[usercol][review_index]
        item_id = ratingsdf[itemcol][review_index]
        rating = ratingsdf[ratingcol][review_index]
        ratings_by_user[user_id][item_id] = rating
        users_by_item[item_id].append(user_id)
    return ratings_by_user,users_by_item

def normalize(data):
    value = data[0]
    mean = data[1]
    std = data[2]
    if std == 0:
        return 0
    else:
        return (value-mean)/std

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0
        
def similarity(ratings1,ratings2):
    common_recipes = set(ratings1.keys()).intersection(set(ratings2.keys()))
    if len(common_recipes)== 0:
        return 0
    l1 = 0
    l2 = 0
    dot = 0
    for recipe_id in common_recipes:
        r1 = ratings1[recipe_id]
        r2 = ratings2[recipe_id]
        l1+=r1**2
        l2+=r2**2
        dot+=r1*r2
    # If the normalized ratings are all 0 for one of the users,
    # the corresponding length will be 0. We may as well return 0.
    if l1*l2 == 0:
        return 0
    else:
        return dot/np.sqrt(l1*l2)

In [3]:
revdf = pd.read_pickle('../data/revdense.pk')

In [6]:
user_ratings,users_by_recipe = get_dictionaries(revdf,'UserId','RecipeId','Rating')
mean_ratings = {user_id:np.mean(list(user_ratings[user_id].values())) for user_id in user_ratings}

In [14]:
revdf['UserMean'] = [mean_ratings[user] for user in revdf.UserId]
revdf['TrueVal'] = (revdf.Rating-revdf.UserMean).apply(sign)

In [16]:
rev_train,rev_test = train_test_split(revdf,test_size=0.2, random_state=1729,stratify = revdf.UserId.values)

In [17]:
user_ratings_train,users_by_recipe_train = get_dictionaries(revdf,'UserId','RecipeId','Rating')
mean_ratings_train = {user_id:np.mean(list(user_ratings_train[user_id].values())) for user_id in user_ratings_train}
std_ratings_train = {user_id:np.std(list(user_ratings_train[user_id].values())) for user_id in user_ratings_train}

In [18]:
rev_train['UserMeanTrain'] = [mean_ratings_train[user] for user in rev_train.UserId]
rev_train['UserStdTrain'] = [std_ratings_train[user] for user in rev_train.UserId]
rev_train['TrainVal'] = (rev_train.Rating-rev_train.UserMean).apply(sign)
rev_train['NormalizedRating'] = [normalize(data) for data in rev_train[['Rating','UserMeanTrain','UserStdTrain']].values]


,UserId,RecipeId,Rating,UserMean,TrueVal,UserMeanTrain,UserStdTrain,TrainVal,NormalizedRating
ReviewId,,,,,,,,,
512403,511821,23425,4,2.000000,1,2.000000,2.000000,1,1.000000
799435,378574,34382,5,3.125000,1,3.125000,2.420615,1,0.774597
175604,67395,113064,5,4.227273,1,4.227273,1.041494,1,0.741941
41353,65308,13380,5,4.428571,1,4.428571,1.026850,1,0.556487
1202780,477308,9989,1,3.684211,-1,3.684211,1.574555,-1,-1.704742
...,...,...,...,...,...,...,...,...,...
174707,71653,56694,0,4.521739,-1,4.521739,1.137104,-1,-3.976539
462066,217482,113930,5,3.900000,1,3.900000,1.920937,1,0.572637
687989,356738,108105,0,3.300000,-1,3.300000,1.791647,-1,-1.841880


In [89]:
def make_prediction(pair,users_dic,recipes_dic,min_sim):
    user_id = pair[0]
    recipe_id = pair[1]
    if recipe_id not in recipes_dic:
        return 'No prediction'
    rating_dic = users_dic[user_id]
    other_recipes = set(rating_dic.keys())
    recipe_reviewers = recipes_dic[recipe_id]
    weights = []
    ratings = []
    for user_id2 in recipe_reviewers:
        rating_dic1 = users_dic[user_id2]
        no_common_recipes = len(set(rating_dic1.keys()).intersection(other_recipes))
        if no_common_recipes>0:
            simscore = similarity(rating_dic,rating_dic1)
            if simscore>= min_sim:
                weights.append(simscore*no_common_recipes)
                ratings.append(rating_dic1[recipe_id])
    if len(ratings)>0:
        return np.average(ratings,weights = weights)
    else:
        return 'No prediction'

In [117]:
rev_test['Pred'] = [make_prediction((rev_test['UserId'][i],rev_test['RecipeId'][i]),
                                    user_ratings_norm,
                                    users_by_recipe,
                                    0.01) for i in rev_test.index]

In [103]:
rev_train['Pred'] = [make_prediction((rev_train['UserId'][i],rev_train['RecipeId'][i]),
                                    user_ratings_norm,
                                    users_by_recipe,
                                    0.2) for i in rev_train.index]

In [115]:
def get_mean_sq_err(pairs):
    return np.mean([(pair[0]-pair[1])**2 for pair in pairs])

In [118]:
get_mean_sq_err(rev_test.loc[rev_test.Pred!='No prediction'][['NormalizedRating','Pred']].values)

4.083552517342434

In [119]:
get_mean_sq_err([[0,x] for x in rev_test.loc[rev_test.Pred!='No prediction']['NormalizedRating']])

3.578167112416195